In [16]:
import pandas as pd
import numpy as np

def topsis(data):
    # Get the numeric columns
    numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()
    # Exclude 'Latitude' and 'Longitude' AND 'cATEGORY id'from the numeric columns
    numeric_columns = [col for col in numeric_columns if col not in ['Latitude', 'Longitude', 'Category ID']]

    # Normalize the decision matrix
    normalized_matrix = data[numeric_columns] / np.linalg.norm(data[numeric_columns], axis=0)

    # Define weights for each criterion
    weights = {
        'Distance': 0.2,
        'Popularity': 0.1,
        'Ratings': 0.3,
        'Total Ratings': 0.1,
        'Price($)': 0.3
    }

    # Ensure the weights match the number of columns in the normalized matrix
    weights_list = [weights[col] for col in numeric_columns]

    # Calculate the weighted normalized decision matrix
    weighted_normalized_matrix = normalized_matrix * np.array(weights_list)

    # Determine the ideal best and ideal worst solutions
    ideal_best = np.max(weighted_normalized_matrix, axis=0)
    ideal_worst = np.min(weighted_normalized_matrix, axis=0)

    # Calculate the distance from the ideal best and ideal worst solutions
    distance_best = np.linalg.norm(weighted_normalized_matrix - ideal_best, axis=1)
    distance_worst = np.linalg.norm(weighted_normalized_matrix - ideal_worst, axis=1)

    # Calculate the performance score
    performance_score = distance_worst / (distance_best + distance_worst)

    # Add the performance score to the DataFrame
    data['Performance Score'] = performance_score

    # Rank the alternatives based on the performance score
    data['Rank'] = data['Performance Score'].rank(ascending=False)
     # Truncate the name
    data['Name'] = data['Name'].apply(lambda x: ' '.join(x.split()[:2]))
    return data[['Name', 'Latitude', 'Longitude', 'Performance Score', 'Rank']]


# Assuming csv_files is a list of file paths
csv_files = ['POI_FINAL_Museum.csv', 'POI_FINAL_Restaurant.csv', 'POI_FINAL_Travel_Agency.csv']

for file in csv_files:
    data = pd.read_csv(file, encoding='latin1')
    rankings = topsis(data)
    print(f"\n{file} Rankings:")
    print(rankings)



POI_FINAL_Museum.csv Rankings:
                Name   Latitude  Longitude  Performance Score  Rank
0            Jane A.  40.767370 -73.981125           0.514328   4.0
1           Spyscape  40.765198 -73.983618           0.374221   8.0
2         The Africa  40.796289 -73.949378           0.618511   1.0
3    Madame Tussauds  40.756539 -73.988521           0.397716   7.0
4           New York  40.752895 -73.977589           0.237920  10.0
5         Center for  40.752768 -73.992304           0.590761   2.0
6  Fashion Institute  40.747304 -73.994797           0.406314   6.0
7       Hebrew Union  40.728807 -73.994549           0.325737   9.0
8         Lower East  40.724864 -73.992735           0.462357   5.0
9            The New  40.726021 -73.999809           0.533399   3.0

POI_FINAL_Restaurant.csv Rankings:
                    Name   Latitude  Longitude  Performance Score  Rank
0       Daily Provisions  40.782225 -73.978646           0.040650  10.0
1          Levain Bakery  40.781511 -73.